# Libraries

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
import pandas as pd
import numpy as np
import sklearn.preprocessing
import sklearn.cross_validation
import sklearn.metrics
import tensorflow
import theano
import keras
import keras.models
import keras.layers
import keras_tqdm

/home/ggomarr/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Couldn't import dot_parser, loading of dot files will not be possible.


Using TensorFlow backend.


# Read data

In [2]:
work_dir='/home/ggomarr/Documents/Education/20170623 Udemy - Machine Learning A-Z: Hands-On Python and R in Data Science/' + \
         'Course data/Part 8 - Deep Learning/Section 39 - Artificial Neural Networks (ANN)/'
df=pd.read_csv(work_dir+'Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
X=df[['Geography','Gender','Age','EstimatedSalary','CreditScore',
      'Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember']].values
Y=df['Exited'].values
X[:5],Y[:5]

(array([['France', 'Female', 42, 101348.88, 619, 2, 0.0, 1, 1, 1],
        ['Spain', 'Female', 41, 112542.58, 608, 1, 83807.86, 1, 0, 1],
        ['France', 'Female', 42, 113931.57, 502, 8, 159660.8, 3, 1, 0],
        ['France', 'Female', 39, 93826.63, 699, 1, 0.0, 2, 0, 0],
        ['Spain', 'Female', 43, 79084.1, 850, 2, 125510.82, 1, 1, 1]], dtype=object),
 array([1, 0, 1, 0, 0]))

# Clean up missing data

In [4]:
sum(X!=X),sum(np.isnan(Y))

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0)

# Create dummy variables

In [5]:
encoder_1=sklearn.preprocessing.LabelEncoder().fit(X[:,0])
X[:,0]=encoder_1.transform(X[:,0])
encoder_2=sklearn.preprocessing.LabelEncoder().fit(X[:,1])
X[:,1]=encoder_2.transform(X[:,1])
one_hot_encoder=sklearn.preprocessing.OneHotEncoder(categorical_features=[0]).fit(X)
X=one_hot_encoder.transform(X).toarray()
X[:5]

array([[  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   4.20000000e+01,   1.01348880e+05,
          6.19000000e+02,   2.00000000e+00,   0.00000000e+00,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   4.10000000e+01,   1.12542580e+05,
          6.08000000e+02,   1.00000000e+00,   8.38078600e+04,
          1.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   4.20000000e+01,   1.13931570e+05,
          5.02000000e+02,   8.00000000e+00,   1.59660800e+05,
          3.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   3.90000000e+01,   9.38266300e+04,
          6.99000000e+02,   1.00000000e+00,   0.00000000e+00,
          2.00000000e+00,   0.00000000e+00,   0.00000000e+00],
    

# Avoid the dummy variable trap

In [6]:
X=np.delete(X,0,1)
X[:5]

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          4.20000000e+01,   1.01348880e+05,   6.19000000e+02,
          2.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          1.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          4.10000000e+01,   1.12542580e+05,   6.08000000e+02,
          1.00000000e+00,   8.38078600e+04,   1.00000000e+00,
          0.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          4.20000000e+01,   1.13931570e+05,   5.02000000e+02,
          8.00000000e+00,   1.59660800e+05,   3.00000000e+00,
          1.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          3.90000000e+01,   9.38266300e+04,   6.99000000e+02,
          1.00000000e+00,   0.00000000e+00,   2.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          4.30

# Feature scaling

In [7]:
X_scaler=sklearn.preprocessing.StandardScaler().fit(X)
X_scaled=X_scaler.transform(X)
X_scaled[:5]

array([[-0.57873591, -0.57380915, -1.09598752,  0.29351742,  0.02188649,
        -0.32622142, -1.04175968, -1.22584767, -0.91158349,  0.64609167,
         0.97024255],
       [-0.57873591,  1.74273971, -1.09598752,  0.19816383,  0.21653375,
        -0.44003595, -1.38753759,  0.11735002, -0.91158349, -1.54776799,
         0.97024255],
       [-0.57873591, -0.57380915, -1.09598752,  0.29351742,  0.2406869 ,
        -1.53679418,  1.03290776,  1.33305335,  2.52705662,  0.64609167,
        -1.03067011],
       [-0.57873591, -0.57380915, -1.09598752,  0.00745665, -0.10891792,
         0.50152063, -1.38753759, -1.22584767,  0.80773656, -1.54776799,
        -1.03067011],
       [-0.57873591,  1.74273971, -1.09598752,  0.38887101, -0.36527578,
         2.06388377, -1.04175968,  0.7857279 , -0.91158349,  0.64609167,
         0.97024255]])

# Train/test split

In [8]:
X_train,X_test,Y_train,Y_test=sklearn.cross_validation.train_test_split(X_scaled,Y,
                                                                        test_size=0.2,random_state=0)
Y_train.size,Y_test.size

(8000, 2000)

# Artificial Neural Networks

## Initialize the network

In [9]:
classifier=keras.models.Sequential()

## Add the layers

In [10]:
classifier.add(keras.layers.Dense(input_dim=11,
                                  units=6,
                                  kernel_initializer='uniform',
                                  activation='relu'))
classifier.add(keras.layers.Dense(units=6,
                                  kernel_initializer='uniform',
                                  activation='relu'))
classifier.add(keras.layers.Dense(units=1,
                                  kernel_initializer='uniform',
                                  activation='sigmoid'))
classifier.get_config()

[{'class_name': 'Dense',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None, 11),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'RandomUniform',
    'config': {'maxval': 0.05, 'minval': -0.05, 'seed': None}},
   'kernel_regularizer': None,
   'name': 'dense_1',
   'trainable': True,
   'units': 6,
   'use_bias': True}},
 {'class_name': 'Dense',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'RandomUniform',
    'config': {'maxval': 0.05, 'minval': -0.05, 'seed': None}},
   'kernel_regularizer': None,
   'name': 'dense_2',
   'trainable': True,
   'units': 6,
   'use_bias

## Compile the network

In [11]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

## Train the network

In [12]:
classifier.fit(X_train,Y_train,batch_size=10,epochs=100,verbose=0,callbacks=[keras_tqdm.TQDMNotebookCallback()])

## All together

In [21]:
def prepare_ANN(structure=[11,6,6,1],transfer='relu',output='sigmoid',
                optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']):
    ann=keras.models.Sequential()
    ann.add(keras.layers.Dense(input_dim=structure[0],
                               units=structure[1],
                               kernel_initializer='uniform',
                               activation=transfer))
    for l in structure[2:-1]:
        ann.add(keras.layers.Dense(units=l,
                                   kernel_initializer='uniform',
                                   activation=transfer))
    ann.add(keras.layers.Dense(units=structure[-1],
                               kernel_initializer='uniform',
                               activation=output))    
    ann.compile(optimizer=optimizer,loss=loss,metrics=metrics)
    return ann

In [35]:
model_classes={ 'ANN':{ 'model':prepare_ANN,
                        'params_pre':{'structure':[11,6,6,1],'transfer':'relu','output':'sigmoid',
                                      'optimizer':'adam','loss':'binary_crossentropy','metrics':['accuracy']},
                        'params_fit':{'batch_size':10,'epochs':25,
                                      'verbose':0,'callbacks':[keras_tqdm.TQDMNotebookCallback()]} },
              }

In [43]:
for class_name in model_classes:
    print('=== {} ==='.format(class_name))
    model_class=model_classes[class_name]['model']
    params_pre=model_classes[class_name]['params_pre']
    params_fit=model_classes[class_name]['params_fit']
    model=model_class(**params_pre).fit(X_train,Y_train,**params_fit)
    print('\nConfusion matrix:')
    print(sklearn.metrics.confusion_matrix(Y_test,model.model.predict_classes(X_test,verbose=0)))
    print('\nClassification report:')
    print(sklearn.metrics.classification_report(Y_test,model.model.predict_classes(X_test,verbose=0)))

=== ANN ===




Confusion matrix:
[[1554   41]
 [ 279  126]]

Classification report:
             precision    recall  f1-score   support

          0       0.85      0.97      0.91      1595
          1       0.75      0.31      0.44       405

avg / total       0.83      0.84      0.81      2000

